In [2]:
#데이터 셋
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

train_data

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


array([[1.23247e+00, 0.00000e+00, 8.14000e+00, ..., 2.10000e+01,
        3.96900e+02, 1.87200e+01],
       [2.17700e-02, 8.25000e+01, 2.03000e+00, ..., 1.47000e+01,
        3.95380e+02, 3.11000e+00],
       [4.89822e+00, 0.00000e+00, 1.81000e+01, ..., 2.02000e+01,
        3.75520e+02, 3.26000e+00],
       ...,
       [3.46600e-02, 3.50000e+01, 6.06000e+00, ..., 1.69000e+01,
        3.62250e+02, 7.83000e+00],
       [2.14918e+00, 0.00000e+00, 1.95800e+01, ..., 1.47000e+01,
        2.61950e+02, 1.57900e+01],
       [1.43900e-02, 6.00000e+01, 2.93000e+00, ..., 1.56000e+01,
        3.76700e+02, 4.38000e+00]])

In [2]:
# 404 훈련 샘플 13개의 특성
train_data.shape

(404, 13)

In [3]:
test_data.shape

(102, 13)

In [1]:
# 주택 가격
train_data

NameError: name 'train_data' is not defined

In [8]:
# 데이터 전처리
# 훈련데이터에서 계산한 값을 사용하여 정규화
# 테스트 데이터나 새로운 데이터도 훈련 데이터에서 사용한 계산 값을 사용해야함

mean = train_data.mean(axis=0)
train_data -= mean

std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std


In [10]:
# 모델 정의
# 훈련 데이터 수가 적을 수록 과대적합이 쉽게 일어나서 작은 모델을 사용하는 것을 추천
from keras import models
from keras import layers

# 동일한 모델을 여러번 생성할 것이므로 함수를 만들어 사용
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1)) # 하나의 유닛에 활서화 함수 없음, 전형적인 스칼라 회귀, 순수한 선형 층, 어떤 범위의 값이라도 예측하도록 냅둠
    model.compile(optimizer='rmsprop', loss='mse',metrics=['mae'])
    return model

In [13]:
# 훈련 검증
# K-겹 검증

import numpy as np

k=4

num_val_samples = len(train_data) // k
print(num_val_samples)
num_epochs = 100
all_scores = []

for i in range(k):
    print('처리중인 폴드 #', i)
    
    # 검증 데이터 준비 : K 번째 분할
    val_data    =    train_data[ i * num_val_samples : (i+1) * num_val_samples]
    val_targets = train_targets[ i * num_val_samples : (i+1) * num_val_samples]
    
    # 훈련 데이터 준비 : 다른 분할 전체
    partial_train_data    = np.concatenate( [   train_data[ : i*num_val_samples],    train_data[(i+1)*num_val_samples : ]], axis=0)
    partial_train_targets = np.concatenate( [train_targets[ : i*num_val_samples], train_targets[(i+1)*num_val_samples : ]], axis=0)
    
    # 모델 구성 및 컴파일
    model = build_model()
    
    # 모델 훈련
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=1, verbose=0)
    
    # 모델 평가 : 검증 세트
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    
    all_scores.append(val_mae)
    

101
처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [14]:
# 각 폴드별 검증 점수
all_scores

[2.525575637817383, 2.4528229236602783, 2.5700507164001465, 2.5373544692993164]

In [16]:
# 최종 평균 점수
np.mean(all_scores)

2.521450936794281

In [18]:
# 최종 모델 훈련하기
# 매개변수 튜닝 후 새롭게 컴파일 된 모델
model = build_model()

# 전체 데이터로 훈련
model.fit(train_data, train_targets, epochs=80, batch_size=16, verbose=0)
    
# 모델 평가 : 검증 세트
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

print('\n')
print(test_mse_score)
print('\n')
print(test_mae_score)

4/4 [==============================] - 0s 1ms/step - loss: 922.7816 - mae: 29.5433


922.7815551757812


29.543319702148438
